In [1]:
import logging
import os
import numpy as np
import pandas as pd

from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, MAX_WORKERS, TEMP_DIR
from src.experiment.GlobalExperiment import GlobalExperiment
from src.experiment.ParhydraExperiment import ParhydraExperiment
from src.experiment.PcitExperiment import PcitExperiment, _Clustering
from src.instance.TSP_Instance import TSP_Instance, TSP_InstanceSet
from src.portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
pd.options.display.max_columns = 999

In [2]:
train_instances, test_instances = TSP_InstanceSet.train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=2,
    seed=0,
)

In [ ]:
t_c = 30
t_v = 30
K = 2
n = 2
solver_class = TSP_LKH_Solver
instance_class= TSP_Instance

experiment = GlobalExperiment(
    t_c=t_c,
    t_v=t_v,
    K=K,
    n=n,
    solver_class=solver_class,
    instance_class=instance_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

[2024-11-15 17:42:07] INFO      [GLOBAL] Start!
[2024-11-15 17:42:07] INFO      Attempt 1/2
[2024-11-15 17:42:07] DEBUG     SMAC configuration, time: [30. 30.]
[2024-11-15 17:42:07] DEBUG     executor start
[2024-11-15 17:42:07] DEBUG     (0, 0) fn submitted
[2024-11-15 17:42:07] DEBUG     (0, 1) fn submitted
[2024-11-15 17:42:07] DEBUG     (1, 0) fn submitted
[2024-11-15 17:42:08] DEBUG     (1, 1) fn submitted
[2024-11-15 17:42:08] DEBUG     (0, 0) result


In [4]:
t_c = 100
t_v = 100
K = 2
n = 2
max_iter = 1
solver_class = TSP_LKH_Solver
instance_class= TSP_Instance

experiment = PcitExperiment(
    t_c=t_c,
    t_v=t_v,
    K=K,
    n=n,
    max_iter=max_iter,
    solver_class=solver_class,
    instance_class=instance_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

[2024-11-14 22:51:27] INFO      [PCIT] Start!
[2024-11-14 22:51:27] INFO      Attempt 1/2
[2024-11-14 22:51:27] INFO      Clustering: {'data/TSP/CEPS_benchmark/cluster_netgen/40.tsp': 0, 'data/TSP/CEPS_benchmark/grid/04.tsp': 1, 'data/TSP/CEPS_benchmark/grid/33.tsp': 0, 'data/TSP/CEPS_benchmark/rotation/45.tsp': 1, 'data/TSP/CEPS_benchmark/uniform_portgen/11.tsp': 0}
[2024-11-14 22:51:27] INFO      Solver 1/2
[2024-11-14 22:51:27] DEBUG     SMAC configuration, time: [100.]
[2024-11-14 22:51:27] DEBUG     executor start
[2024-11-14 22:51:34] DEBUG     data/TSP/CEPS_benchmark/cluster_netgen/40.tsp features calculated
[2024-11-14 22:51:35] DEBUG     data/TSP/CEPS_benchmark/grid/33.tsp features calculated
[2024-11-14 22:51:35] DEBUG     data/TSP/CEPS_benchmark/uniform_portgen/11.tsp features calculated
[2024-11-14 22:51:35] DEBUG     (0, 0) fn submitted
[2024-11-14 22:51:35] DEBUG     (1, 0) fn submitted
[2024-11-14 22:51:35] DEBUG     (2, 0) fn submitted
[2024-11-14 22:51:35] DEBUG     (0

In [ ]:
# t_c = 30
# t_v = 30
# K = 2
# n = 3
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance

# experiment = ParhydraExperiment(
#     t_c=t_c,
#     t_v=t_v,
#     K=K,
#     n=n,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
remaining_time = np.ones(shape=(K, )) * np.inf
best_portfolio.evaluate(test_instances, remaining_time, "test")

In [5]:
from src.database import db_connect
conn = db_connect()

In [2]:
import sqlite3
conn = sqlite3.connect("database/2024_11_14_20_51_33.db")

In [6]:
df_solvers = pd.read_sql_query("SELECT * FROM solvers", conn)
df_solvers

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,INITIAL_TOUR_ALGORITHM,INITIAL_TOUR_FRACTION,KICKS,KICK_TYPE,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,1359926245157039021,42,1,YES,DELAUNAY,6,QUADRANT,NO,NO,2,GREEDY,0.251639,2,0,296366314,6,4,0,0,54,YES,YES,3,YES
1,696333515371916119,59,0,YES,ALPHA,10,QUADRANT,YES,NO,2,GREEDY,0.978027,4,0,1559387156,10,2,3,4,71,YES,NO,6,YES
2,439035582680904907,57,0,NO,NEAREST-NEIGHBOR,1,QUADRANT,YES,YES,2,BORUVKA,0.589894,3,0,845145976,1,4,2,5,34,YES,NO,0,NO
3,1829562326984185848,50,0,YES,QUADRANT,7,QUADRANT,NO,YES,4,WALK,0.072133,4,0,550810371,10,4,3,0,22,NO,YES,0,YES
4,76062450504648900,60,0,NO,QUADRANT,7,QUADRANT,YES,NO,2,GREEDY,0.367404,0,0,972489772,5,4,3,4,28,YES,YES,4,NO
5,1511101017643655546,56,1,NO,NEAREST-NEIGHBOR,8,QUADRANT,NO,NO,2,QUICK-BORUVKA,0.900395,4,4,1623274060,5,5,1,5,78,YES,YES,3,NO
6,391084944223481301,53,1,YES,ALPHA,0,QUADRANT,YES,YES,5,NEAREST-NEIGHBOR,0.188252,5,5,1207107773,5,5,1,5,22,YES,NO,0,NO
7,1438539737709382543,41,1,NO,NEAREST-NEIGHBOR,2,QUADRANT,NO,YES,3,BORUVKA,0.745781,3,0,3195185,2,3,1,0,17,YES,NO,4,NO
8,1034895483521450110,57,0,NO,ALPHA,8,QUADRANT,YES,NO,4,BORUVKA,0.427505,5,5,1928112859,3,2,5,5,6,YES,YES,4,NO
9,1855047126785556270,40,0,YES,NEAREST-NEIGHBOR,7,QUADRANT,NO,YES,1,NEAREST-NEIGHBOR,0.035908,4,0,226458313,1,3,0,3,91,NO,NO,4,YES


In [4]:
df_instances = pd.read_sql_query("SELECT * FROM instances", conn)
df_instances

,id
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp
1,data/TSP/CEPS_benchmark/grid/04.tsp
2,data/TSP/CEPS_benchmark/grid/33.tsp
3,data/TSP/CEPS_benchmark/rotation/45.tsp
4,data/TSP/CEPS_benchmark/uniform_portgen/11.tsp
...,...
495,data/TSP/CEPS_benchmark/implosion/23.tsp
496,data/TSP/CEPS_benchmark/expansion/42.tsp
497,data/TSP/CEPS_benchmark/compression/17.tsp
498,data/TSP/CEPS_benchmark/cluster/47.tsp


In [5]:
df = pd.read_sql_query("SELECT * FROM results", conn)
# df.to_excel("tmp.xlsx")
df

,instance_id,solver_id,cost,time,comment,created_at
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,2.56,2.56,configuration,2024-11-14 19:51:36
1,data/TSP/CEPS_benchmark/grid/04.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,1.52,1.52,configuration,2024-11-14 19:51:36
2,data/TSP/CEPS_benchmark/grid/33.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,100.00,10.00,configuration,2024-11-14 19:51:44
3,data/TSP/CEPS_benchmark/rotation/45.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,1.62,1.62,configuration,2024-11-14 19:51:44
4,data/TSP/CEPS_benchmark/uniform_portgen/11.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,100.00,10.00,configuration,2024-11-14 19:51:44
...,...,...,...,...,...,...
14466,data/TSP/CEPS_benchmark/cluster/47.tsp,pGMaf_Q0_jgn1a8RuQOPm-JbG56DfxICXU9MatpxGnk=,100.00,10.00,test,2024-11-15 04:49:17
14467,data/TSP/CEPS_benchmark/expansion/22.tsp,Fz2zAidGUC32x-pUWGK9luiIeWBwGTBuLSy5SVy8WWw=,100.00,10.00,test,2024-11-15 04:49:21
14468,data/TSP/CEPS_benchmark/expansion/22.tsp,1nVn_uaKMu4KDgigbRGffCMyQ5D2XpuKaQ80b_b6lwo=,100.00,10.00,test,2024-11-15 04:49:21
14469,data/TSP/CEPS_benchmark/expansion/22.tsp,00fvonqWrw52Qc679afbPxrng7wwPxvHQ5LtIJ4H7gA=,100.00,10.00,test,2024-11-15 04:49:21


In [6]:
df.loc[df["comment"] == "configuration"]

,instance_id,solver_id,cost,time,comment,created_at
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,2.56,2.56,configuration,2024-11-14 19:51:36
1,data/TSP/CEPS_benchmark/grid/04.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,1.52,1.52,configuration,2024-11-14 19:51:36
2,data/TSP/CEPS_benchmark/grid/33.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,100.00,10.00,configuration,2024-11-14 19:51:44
3,data/TSP/CEPS_benchmark/rotation/45.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,1.62,1.62,configuration,2024-11-14 19:51:44
4,data/TSP/CEPS_benchmark/uniform_portgen/11.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,100.00,10.00,configuration,2024-11-14 19:51:44
...,...,...,...,...,...,...
12586,data/TSP/CEPS_benchmark/cluster/15.tsp,KNyqGNpvCRs_eaMnPo5Cexsg_TfeQ9N5-oErntVSsEE=,100.00,10.00,configuration,2024-11-15 03:49:02
12587,data/TSP/CEPS_benchmark/implosion/16.tsp,KNyqGNpvCRs_eaMnPo5Cexsg_TfeQ9N5-oErntVSsEE=,100.00,10.00,configuration,2024-11-15 03:49:02
12588,data/TSP/CEPS_benchmark/uniform_portgen/39.tsp,KNyqGNpvCRs_eaMnPo5Cexsg_TfeQ9N5-oErntVSsEE=,100.00,10.00,configuration,2024-11-15 03:49:02
12589,data/TSP/CEPS_benchmark/expansion/09.tsp,KNyqGNpvCRs_eaMnPo5Cexsg_TfeQ9N5-oErntVSsEE=,100.00,10.00,configuration,2024-11-15 03:49:03


In [7]:
_ = df.loc[df["comment"] == "test"].pivot_table(index="instance_id", columns="solver_id", values="cost")
_["best"] = np.argmin(_.values, axis=1)
_["min"] = _.iloc[:, :-1].min(axis=1)
_

solver_id,00fvonqWrw52Qc679afbPxrng7wwPxvHQ5LtIJ4H7gA=,1nVn_uaKMu4KDgigbRGffCMyQ5D2XpuKaQ80b_b6lwo=,Fz2zAidGUC32x-pUWGK9luiIeWBwGTBuLSy5SVy8WWw=,pGMaf_Q0_jgn1a8RuQOPm-JbG56DfxICXU9MatpxGnk=,best,min
instance_id,,,,,,
data/TSP/CEPS_benchmark/cluster/00.tsp,100.00,100.0,100.0,100.0,0,100.00
data/TSP/CEPS_benchmark/cluster/01.tsp,100.00,100.0,100.0,100.0,0,100.00
data/TSP/CEPS_benchmark/cluster/02.tsp,100.00,100.0,100.0,100.0,0,100.00
data/TSP/CEPS_benchmark/cluster/03.tsp,4.39,100.0,100.0,100.0,0,4.39
data/TSP/CEPS_benchmark/cluster/04.tsp,100.00,100.0,100.0,100.0,0,100.00
...,...,...,...,...,...,...
data/TSP/CEPS_benchmark/uniform_portgen/45.tsp,0.61,100.0,100.0,100.0,0,0.61
data/TSP/CEPS_benchmark/uniform_portgen/46.tsp,3.47,100.0,100.0,100.0,0,3.47
data/TSP/CEPS_benchmark/uniform_portgen/47.tsp,1.11,100.0,100.0,100.0,0,1.11


In [10]:
_["min"].describe()

count    470.000000
mean      42.247596
std       47.665419
min        0.300000
25%        1.665000
50%        6.145000
75%      100.000000
max      100.000000
Name: min, dtype: float64

In [12]:
(_["min"] > 90).value_counts()

False    280
True     190
Name: min, dtype: int64

In [ ]:
_.loc[lambda x: x["min"] < 90, "best"].value_counts()

In [ ]:
_.sort_values(by="min")

In [ ]:
config = df_solvers.loc[df_solvers["id"] == "OJQGNYe__-omEPLuUJkSzOuFdCnYPLzZd99VQqxW2gM="].drop(columns=["id"]).iloc[0].to_dict()
config = Configuration(configuration_space=CONFIGURATION_SPACE, values=config)
solver = TSP_LKH_Solver(config=config)

instance = TSP_Instance(filepath="data/TSP/CEPS_benchmark/cluster_netgen/18.tsp", optimum=10555222.0)

solver.solve(instance)

In [ ]:
# import json
# with open(DATA_DIR / "TSP" / "index.json") as f:
#     index = json.load(f)

# new_index = {}

# for k, v in index.items():
#     path = k.split("/")
#     file = path[-1]
#     file_no_ext = file.split(".")[0]
#     if "_" in file_no_ext:
#         a, b = file_no_ext.split("_")
#         n = str((int(a) - 4) * 10 + int(b)).zfill(2)
#         new_file = f"{n}.tsp"
#     else:
#         file_no_ext = str(int(file_no_ext)).zfill(2)
#         new_file = f"{file_no_ext}.tsp"
#     new_path = path[:-1] + [new_file]
#     new_k = "/".join(new_path)
#     new_index[new_k] = float(v)

#     path1 = DATA_DIR / k;
#     path2 = DATA_DIR / new_k;
#     path1.rename(path2)

# new_index = {k: new_index[k] for k in sorted(new_index.keys())}

# with open(DATA_DIR / "TSP" / "index.json", "w") as f:
#     json.dump(new_index, f, indent=4)

# new_index